<a href="https://colab.research.google.com/github/Jahnvi99/qWiki-Search/blob/main/qWiki_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11697 sha256=52cecebb9e7f0a2849e8e6fb707bbcb9586ca831cafa727695036137269616e4
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


#Importing libraries

In [2]:
import re
import nltk
import numpy as np
import wikipedia
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Finding relevant wikipedia pages

In [3]:
query = input('Enter query: ')
n = int(input('Enter number of sentences to be returned: '))

titles = wikipedia.search(query, results=5)
if not titles:
	print ('The given query does not match any wikipedia page.')
	exit()

Enter query: Mahatma Gandhi
Enter number of sentences to be returned: 7


#Extracting temporal sentences based only on year

In [4]:
data = []
for title in titles:
    try:
        raw_data = wikipedia.page(title).content.replace('\n', ' ')
        processed_data = re.sub('== References ==.+|== See Also ==.+', '', raw_data) #removing irrelevant data from content
        data.append(processed_data)
    except Exception:
        pass

articles = [nltk.sent_tokenize(text) for text in data]
temporal_sentences = []
for article in articles:
    for sentence in article:
        if re.search(r'\d{4}', sentence) is not None:               
            temporal_sentences.append(re.sub('=+.+?=+', '', sentence)) #removing all headings and subheadings

#Creating the document word matrix A

In [5]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(temporal_sentences)

#Creating the model to calculate A=WH

In [6]:
nmf_model = NMF(n_components=1, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
W = nmf_model.transform(tfidf)

#Selecting top documents based on the score from topic to document matrix(W)

In [7]:
top_sent_indices = np.argsort( W[:,0] )[::-1][0:n]
best_sentences = [temporal_sentences[sent_index] for sent_index in top_sent_indices]

#Sort and print the final output




In [8]:
output = []
for line in best_sentences:
    year = re.search(r'\d{4}', line).group()
    output.append((year, line))
sorted_output = sorted(output, key=lambda tup: tup[0], reverse = True)
for item in sorted_output:
	print(item)

('1948', 'Mahatma Gandhi was assassinated on 30 January 1948 in the compound of Birla Gandhi’s House (now Gandhi Smriti), a large mansion in New Delhi.')
('1948', "Among them was Nathuram Godse, a Hindu nationalist who assassinated Gandhi on 30 January 1948 by firing three bullets into his chest.Gandhi's birthday, 2 October, is commemorated in India as Gandhi Jayanti, a national holiday, and worldwide as the International Day of Nonviolence.")
('1930', "Gandhi led Congress in a  celebration on 26 January 1930 of India's Independence Day in Lahore.")
('1920', "Gandhi took Gokhale's liberal approach based on British Whiggish traditions and transformed it to make it look Indian.Gandhi took leadership of the Congress in 1920 and began escalating demands until on 26 January 1930 the Indian National Congress declared the independence of India.")
('1888', 'Harilal Mohandas Gandhi (23 August 1888 – 18 June 1948) was the eldest son of Mohandas Karamchand Gandhi and Kasturba Gandhi.')
('1869', '